<a href="https://colab.research.google.com/github/oraziotorre/TTinsight/blob/main/TTinsight_DataPreprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [247]:
import pandas as pd
import ast
from collections import Counter
from google.colab import data_table
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Concatenate, Embedding, Dropout

data_table.enable_dataframe_formatter()

In [248]:
dataset = pd.read_csv("raw_dataset.csv")

dataset

,event_id,match_id,match_format,players_gender,match_stage,stage_id,match_duration,match_start_time,player_id,player_2_id,opponent_id,opponent_2_id,player_sets_won,opponent_sets_won,match_scores,sets_required_to_win,current_match_state,player_points,opponent_points,match_result
0,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,FNL,100,1990,2021-11-15 17:32:48,112463,115009.0,117294,109777.0,3,2,"8-11,11-8,11-9,7-11,11-8",3,0-0,"[0, 0, 1, 2, 2, 3, 4, 4, 4, 5, 6, 6, 6, 6, 6, ...","[1, 2, 2, 2, 3, 3, 3, 4, 5, 5, 5, 6, 7, 8, 9, ...",W
1,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,FNL,100,1990,2021-11-15 17:32:48,117294,109777.0,112463,115009.0,2,3,"11-8,8-11,9-11,11-7,8-11",3,0-0,"[1, 2, 2, 2, 3, 3, 3, 4, 5, 5, 5, 6, 7, 8, 9, ...","[0, 0, 1, 2, 2, 3, 4, 4, 4, 5, 6, 6, 6, 6, 6, ...",L
2,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,FNL,100,1990,2021-11-15 17:32:48,112463,115009.0,117294,109777.0,3,2,"8-11,11-8,11-9,7-11,11-8",3,0-1,"[1, 1, 1, 1, 2, 3, 4, 5, 6, 7, 7, 8, 9, 9, 9, ...","[0, 1, 2, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 5, 6, ...",W
3,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,FNL,100,1990,2021-11-15 17:32:48,117294,109777.0,112463,115009.0,2,3,"11-8,8-11,9-11,11-7,8-11",3,1-0,"[0, 1, 2, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 5, 6, ...","[1, 1, 1, 1, 2, 3, 4, 5, 6, 7, 7, 8, 9, 9, 9, ...",L
4,2234,TTEWDOUBLES-----------FNL-000100----------,D,W,FNL,100,1990,2021-11-15 17:32:48,112463,115009.0,117294,109777.0,3,2,"8-11,11-8,11-9,7-11,11-8",3,1-1,"[1, 1, 2, 2, 2, 3, 3, 4, 4, 5, 6, 6, 6, 6, 7, ...","[0, 1, 1, 2, 3, 3, 4, 4, 5, 5, 5, 6, 7, 8, 8, ...",W
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144189,2997,TTEMSINGLES-----------RND2000600----------,S,M,RND2,600,2452,2024-10-29 13:45:00,137238,NaN,132172,NaN,3,2,"11-3,7-11,11-9,7-11,11-6",3,1-1,"[0, 1, 1, 2, 2, 3, 4, 4, 4, 5, 5, 6, 6, 7, 8, ...","[1, 1, 2, 2, 3, 3, 3, 4, 5, 5, 6, 6, 7, 7, 7, ...",W
144190,2997,TTEMSINGLES-----------RND2000600----------,S,M,RND2,600,2452,2024-10-29 13:45:00,132172,NaN,137238,NaN,2,3,"3-11,11-7,9-11,11-7,6-11",3,1-2,"[1, 1, 2, 3, 3, 4, 5, 6, 6, 6, 7, 7, 7, 7, 8, ...","[0, 1, 1, 1, 2, 2, 2, 2, 3, 4, 4, 5, 6, 7, 7, ...",L
144191,2997,TTEMSINGLES-----------RND2000600----------,S,M,RND2,600,2452,2024-10-29 13:45:00,137238,NaN,132172,NaN,3,2,"11-3,7-11,11-9,7-11,11-6",3,2-1,"[0, 1, 1, 1, 2, 2, 2, 2, 3, 4, 4, 5, 6, 7, 7, ...","[1, 1, 2, 3, 3, 4, 5, 6, 6, 6, 7, 7, 7, 7, 8, ...",W
144192,2997,TTEMSINGLES-----------RND2000600----------,S,M,RND2,600,2452,2024-10-29 13:45:00,132172,NaN,137238,NaN,2,3,"3-11,11-7,9-11,11-7,6-11",3,2-2,"[1, 2, 2, 3, 3, 3, 3, 3, 3, 3, 4, 5, 5, 5, 6, 6]","[0, 0, 1, 1, 2, 3, 4, 5, 6, 7, 7, 7, 8, 9, 9, 10]",L


# **Pulizia RawDataset**
Rimozione delle righe e delle colonne indesiderate

In [249]:
nan_mask = dataset.isna()
nan_count = nan_mask.sum()

# Verifico i valori null del raw_dataset
nan_count


,0
event_id,0
match_id,0
match_format,0
players_gender,0
match_stage,0
stage_id,0
match_duration,0
match_start_time,0
player_id,0
player_2_id,109078


In [250]:
# Tolgo tutti i set con punteggi nulli
dataset_cleaned = dataset.dropna(subset=["player_points"])
dataset_cleaned[dataset_cleaned['player_points'].isnull()]

,event_id,match_id,match_format,players_gender,match_stage,stage_id,match_duration,match_start_time,player_id,player_2_id,opponent_id,opponent_2_id,player_sets_won,opponent_sets_won,match_scores,sets_required_to_win,current_match_state,player_points,opponent_points,match_result


In [251]:
# Verifico la correttezza dei campi numerici
dataset_cleaned.describe()

,event_id,stage_id,match_duration,player_id,player_2_id,opponent_id,opponent_2_id,player_sets_won,opponent_sets_won,sets_required_to_win
count,139214.000000,139214.000000,139214.000000,139214.000000,33754.000000,139214.000000,33754.000000,139214.000000,139214.000000,139214.000000
mean,2702.913687,1549.561984,2045.959530,123822.997062,125763.463738,123822.997062,125763.463738,2.085099,2.085099,3.146321
std,156.759090,5440.368764,1410.717125,17170.652931,17745.832474,17170.652931,17745.832474,1.251598,1.251598,0.353551
min,2234.000000,100.000000,0.000000,90051.000000,100001.000000,90051.000000,100001.000000,0.000000,0.000000,2.000000
25%,2569.000000,200.000000,1422.000000,114715.000000,115565.000000,114715.000000,115565.000000,1.000000,1.000000,3.000000
50%,2701.000000,600.000000,1912.000000,121403.000000,122428.000000,121403.000000,122428.000000,3.000000,3.000000,3.000000
75%,2869.000000,1200.000000,2427.000000,132028.000000,133124.000000,132028.000000,133124.000000,3.000000,3.000000,3.000000
max,2997.000000,110005.000000,77074.000000,212088.000000,212090.000000,212088.000000,212090.000000,5.000000,5.000000,5.000000


In [252]:
# Rimuovo le righe contenenti set con punteggi rari che non voglio considerare
dataset_cleaned = dataset_cleaned[~dataset_cleaned['sets_required_to_win'].isin([2, 5])]

dataset_cleaned.describe()

,event_id,stage_id,match_duration,player_id,player_2_id,opponent_id,opponent_2_id,player_sets_won,opponent_sets_won,sets_required_to_win
count,139208.000000,139208.000000,139208.000000,139208.000000,33754.000000,139208.000000,33754.000000,139208.000000,139208.000000,139208.000000
mean,2702.922073,1549.569867,2045.960505,123823.268735,125763.463738,123823.268735,125763.463738,2.085110,2.085110,3.146328
std,156.757252,5440.484808,1410.737123,17170.898007,17745.832474,17170.898007,17745.832474,1.251576,1.251576,0.353436
min,2234.000000,100.000000,0.000000,90051.000000,100001.000000,90051.000000,100001.000000,0.000000,0.000000,3.000000
25%,2569.000000,200.000000,1422.000000,114715.000000,115565.000000,114715.000000,115565.000000,1.000000,1.000000,3.000000
50%,2701.000000,600.000000,1912.000000,121403.000000,122428.000000,121403.000000,122428.000000,3.000000,3.000000,3.000000
75%,2869.000000,1200.000000,2427.000000,132028.000000,133124.000000,132028.000000,133124.000000,3.000000,3.000000,3.000000
max,2997.000000,110005.000000,77074.000000,212088.000000,212090.000000,212088.000000,212090.000000,4.000000,4.000000,4.000000


Verifico la lunghezza dei punteggi per ogni riga

In [253]:
sequence_lengths = dataset_cleaned['player_points'].apply(lambda points: len(points.split(',')))

# Contiamo quante righe corrispondono a ciascuna lunghezza
length_counts = sequence_lengths.value_counts().sort_index()

# Stampiamo il risultato
print("Distribuzione delle lunghezze delle sequenze:")
print(length_counts)

Distribuzione delle lunghezze delle sequenze:
player_points
10      204
11     1578
12     3818
13     7058
14    10902
15    14748
16    18042
17    20078
18    21106
19    21192
21    10198
23     5152
25     2560
27     1238
29      698
31      298
33      136
35      106
37       44
39       24
41       16
43        8
45        4
Name: count, dtype: int64


Elimino i set rari in cui hanno giocato più di 30 punti(metto 29 perchè nel dataset non considero il punto finale di vittoria della partita)

In [269]:
sequence_lengths = dataset_cleaned['player_points'].apply(lambda points: len(points.split(',')))

# Filtra il DataFrame per mantenere solo le righe con sequenze di lunghezza <= 29
dataset_cleaned = dataset_cleaned[sequence_lengths <= 29]

sequence_lengths = dataset_cleaned['player_points'].apply(lambda points: len(points.split(',')))

# Contiamo quante righe corrispondono a ciascuna lunghezza
length_counts = sequence_lengths.value_counts().sort_index()

# Stampiamo il risultato
print("Distribuzione delle lunghezze delle sequenze:")
print(length_counts)

Distribuzione delle lunghezze delle sequenze:
player_points
10      204
11     1578
12     3818
13     7058
14    10902
15    14748
16    18042
17    20078
18    21106
19    21192
21    10198
23     5152
25     2560
27     1238
29      698
Name: count, dtype: int64


Elimino le colonne che non mi interessano

In [255]:
columns_to_drop = ['event_id', 'match_id', 'stage_id', 'match_duration',
                  'match_start_time','opponent_id', 'opponent_2_id',
                  'player_sets_won', 'opponent_sets_won', 'match_scores',
                  'opponent_points']

# Rimuovo le colonne che non mi interessano
dataset_cleaned = dataset_cleaned.drop(columns=columns_to_drop, axis=1)

dataset_cleaned

,match_format,players_gender,match_stage,player_id,player_2_id,sets_required_to_win,current_match_state,player_points,match_result
0,D,W,FNL,112463,115009.0,3,0-0,"[0, 0, 1, 2, 2, 3, 4, 4, 4, 5, 6, 6, 6, 6, 6, ...",W
1,D,W,FNL,117294,109777.0,3,0-0,"[1, 2, 2, 2, 3, 3, 3, 4, 5, 5, 5, 6, 7, 8, 9, ...",L
2,D,W,FNL,112463,115009.0,3,0-1,"[1, 1, 1, 1, 2, 3, 4, 5, 6, 7, 7, 8, 9, 9, 9, ...",W
3,D,W,FNL,117294,109777.0,3,1-0,"[0, 1, 2, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 5, 6, ...",L
4,D,W,FNL,112463,115009.0,3,1-1,"[1, 1, 2, 2, 2, 3, 3, 4, 4, 5, 6, 6, 6, 6, 7, ...",W
...,...,...,...,...,...,...,...,...,...
144189,S,M,RND2,137238,NaN,3,1-1,"[0, 1, 1, 2, 2, 3, 4, 4, 4, 5, 5, 6, 6, 7, 8, ...",W
144190,S,M,RND2,132172,NaN,3,1-2,"[1, 1, 2, 3, 3, 4, 5, 6, 6, 6, 7, 7, 7, 7, 8, ...",L
144191,S,M,RND2,137238,NaN,3,2-1,"[0, 1, 1, 1, 2, 2, 2, 2, 3, 4, 4, 5, 6, 7, 7, ...",W
144192,S,M,RND2,132172,NaN,3,2-2,"[1, 2, 2, 3, 3, 3, 3, 3, 3, 3, 4, 5, 5, 5, 6, 6]",L


#**Operazione sulle feature**


Facciamo alcune considerazioni sulle caratteristiche del nostro dataset e identifichiamo possibili miglioramenti per ottimizzarlo in vista del prossimo training del modello:

* Le colonne *match_format*, *player_id*, *player_2_id* non sono necessarie per il training del modello e possono essere cancellate

* Le colonne *sets_required_to_win* e *current_match_state* fanno entrambe riferimento al numero di set necessari per vincere una partita. Per rendere il dataset più chiaro e organizzato, possiamo sostituirle con due nuove colonne: *sets_to_win* e *opponent_sets_to_win*, che rappresentano rispettivamente il numero di set necessari al giocatore e all'avversario per vincere la partita.

* La colonna *match_stage* può essere modificata per attribuire maggiore importanza alle partite finali (FNL) rispetto a tutte le altre, migliorando il peso di queste informazioni nel training del modello.

* Per la colonna *match_result* si può semplificare il formato binario e facilitare l'apprendimento del modello.

* Per la colonna *players_gender* si può semplificare il formato tramite l'operazione di *One-Hot Encoding* nel caso di dataset_all_matches o tramite una semplice codifica binaria nel caso del dataset_single_matches

##**Colonne *sets_required_to_win* e *current_match_state***
Il principale vantaggio nell'utilizzare *sets_to_win* e *opponent_sets_to_win* invece delle colonne originali è la semplicità. Queste nuove variabili contengono le informazioni cruciali per il modello, senza introdurre troppa complessità. La loro adozione facilita la gestione dei dati, migliorando l'efficienza del modello, **riassumendo le qualità utili delle colonne originarie** (*set_required_to_win* e *current_match_state*)

In [256]:
# Splittiamo 'current_match_state' e 'sets_to_win' in due nuove colonne
dataset_cleaned['sets_to_win'] = dataset_cleaned['sets_required_to_win'] - dataset_cleaned['current_match_state'].str.split('-', expand=True)[0].astype(int)
dataset_cleaned['opponent_sets_to_win'] = dataset_cleaned['sets_required_to_win'] - dataset_cleaned['current_match_state'].str.split('-', expand=True)[1].astype(int)
dataset_cleaned.drop(['sets_required_to_win', 'current_match_state'], axis=1, inplace=True)

# Riorganizziamo l'ordine delle colonne
match_stage_index = dataset_cleaned.columns.get_loc('match_stage')
set_diff_column_1 = dataset_cleaned.pop('sets_to_win')
set_diff_column_2 = dataset_cleaned.pop('opponent_sets_to_win')
dataset_cleaned.insert(match_stage_index + 1, 'sets_to_win', set_diff_column_1)
dataset_cleaned.insert(match_stage_index + 2, 'opponent_sets_to_win', set_diff_column_2)

# Visualizziamo il dataset_cleaned aggiornato
dataset_cleaned


,match_format,players_gender,match_stage,sets_to_win,opponent_sets_to_win,player_id,player_2_id,player_points,match_result
0,D,W,FNL,3,3,112463,115009.0,"[0, 0, 1, 2, 2, 3, 4, 4, 4, 5, 6, 6, 6, 6, 6, ...",W
1,D,W,FNL,3,3,117294,109777.0,"[1, 2, 2, 2, 3, 3, 3, 4, 5, 5, 5, 6, 7, 8, 9, ...",L
2,D,W,FNL,3,2,112463,115009.0,"[1, 1, 1, 1, 2, 3, 4, 5, 6, 7, 7, 8, 9, 9, 9, ...",W
3,D,W,FNL,2,3,117294,109777.0,"[0, 1, 2, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 5, 6, ...",L
4,D,W,FNL,2,2,112463,115009.0,"[1, 1, 2, 2, 2, 3, 3, 4, 4, 5, 6, 6, 6, 6, 7, ...",W
...,...,...,...,...,...,...,...,...,...
144189,S,M,RND2,2,2,137238,NaN,"[0, 1, 1, 2, 2, 3, 4, 4, 4, 5, 5, 6, 6, 7, 8, ...",W
144190,S,M,RND2,2,1,132172,NaN,"[1, 1, 2, 3, 3, 4, 5, 6, 6, 6, 7, 7, 7, 7, 8, ...",L
144191,S,M,RND2,1,2,137238,NaN,"[0, 1, 1, 1, 2, 2, 2, 2, 3, 4, 4, 5, 6, 7, 7, ...",W
144192,S,M,RND2,1,1,132172,NaN,"[1, 2, 2, 3, 3, 3, 3, 3, 3, 3, 4, 5, 5, 5, 6, 6]",L


##**Colonna *match_stage***

Al posto della colonna *match_stage* inseriamo *is_final* in modo da semplificarne i valori in:
- **FNL = 1**
- **Tutti gli altri stage del torneo = 0**

In questo modo possiamo dare maggiore importanza alle partite finali (FNL) rispetto a tutte le altre, **migliorando il peso di queste informazioni** nel training del modello.

In [257]:
# Creiamo la colonna 'is_final' il cui valore è 1 se 'match_stage' == 'FNL', altrimenti 0
dataset_cleaned['is_final'] = (dataset_cleaned['match_stage'] == 'FNL').astype(int)

# Rimuoviamo la colonna 'match_stage'
dataset_cleaned = dataset_cleaned.drop(columns=['match_stage'])

# Posizioniamo 'is_final' dopo 'players_gender'
dataset_cleaned = dataset_cleaned[['players_gender', 'is_final'] + [col for col in dataset_cleaned.columns if col not in ['players_gender', 'is_final']]]

dataset_cleaned

,players_gender,is_final,match_format,sets_to_win,opponent_sets_to_win,player_id,player_2_id,player_points,match_result
0,W,1,D,3,3,112463,115009.0,"[0, 0, 1, 2, 2, 3, 4, 4, 4, 5, 6, 6, 6, 6, 6, ...",W
1,W,1,D,3,3,117294,109777.0,"[1, 2, 2, 2, 3, 3, 3, 4, 5, 5, 5, 6, 7, 8, 9, ...",L
2,W,1,D,3,2,112463,115009.0,"[1, 1, 1, 1, 2, 3, 4, 5, 6, 7, 7, 8, 9, 9, 9, ...",W
3,W,1,D,2,3,117294,109777.0,"[0, 1, 2, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 5, 6, ...",L
4,W,1,D,2,2,112463,115009.0,"[1, 1, 2, 2, 2, 3, 3, 4, 4, 5, 6, 6, 6, 6, 7, ...",W
...,...,...,...,...,...,...,...,...,...
144189,M,0,S,2,2,137238,NaN,"[0, 1, 1, 2, 2, 3, 4, 4, 4, 5, 5, 6, 6, 7, 8, ...",W
144190,M,0,S,2,1,132172,NaN,"[1, 1, 2, 3, 3, 4, 5, 6, 6, 6, 7, 7, 7, 7, 8, ...",L
144191,M,0,S,1,2,137238,NaN,"[0, 1, 1, 1, 2, 2, 2, 2, 3, 4, 4, 5, 6, 7, 7, ...",W
144192,M,0,S,1,1,132172,NaN,"[1, 2, 2, 3, 3, 3, 3, 3, 3, 3, 4, 5, 5, 5, 6, 6]",L


##**Colonna *match_results***
Convertiamo la colonna *match_result*, originariamente composta dai valori W (Win) e L (Loss), in una feature binaria con i valori 1 (Win) e 0 (Loss), semplificando il formato e facilitando l'apprendimento del modello.

In [258]:
# Creiamo la colonna 'is_winner' il cui valore è 1 se 'match_result' == 'W', altrimenti 0
dataset_cleaned['is_winner'] = (dataset_cleaned['match_result'] == 'W').astype(int)

#Rimuoviamo la colonna 'match_result' e non abbiamo neanche bisogno di riordinare il database
dataset_cleaned = dataset_cleaned.drop(columns = ['match_result'])

dataset_cleaned

,players_gender,is_final,match_format,sets_to_win,opponent_sets_to_win,player_id,player_2_id,player_points,is_winner
0,W,1,D,3,3,112463,115009.0,"[0, 0, 1, 2, 2, 3, 4, 4, 4, 5, 6, 6, 6, 6, 6, ...",1
1,W,1,D,3,3,117294,109777.0,"[1, 2, 2, 2, 3, 3, 3, 4, 5, 5, 5, 6, 7, 8, 9, ...",0
2,W,1,D,3,2,112463,115009.0,"[1, 1, 1, 1, 2, 3, 4, 5, 6, 7, 7, 8, 9, 9, 9, ...",1
3,W,1,D,2,3,117294,109777.0,"[0, 1, 2, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 5, 6, ...",0
4,W,1,D,2,2,112463,115009.0,"[1, 1, 2, 2, 2, 3, 3, 4, 4, 5, 6, 6, 6, 6, 7, ...",1
...,...,...,...,...,...,...,...,...,...
144189,M,0,S,2,2,137238,NaN,"[0, 1, 1, 2, 2, 3, 4, 4, 4, 5, 5, 6, 6, 7, 8, ...",1
144190,M,0,S,2,1,132172,NaN,"[1, 1, 2, 3, 3, 4, 5, 6, 6, 6, 7, 7, 7, 7, 8, ...",0
144191,M,0,S,1,2,137238,NaN,"[0, 1, 1, 1, 2, 2, 2, 2, 3, 4, 4, 5, 6, 7, 7, ...",1
144192,M,0,S,1,1,132172,NaN,"[1, 2, 2, 3, 3, 3, 3, 3, 3, 3, 4, 5, 5, 5, 6, 6]",0


In [259]:
#Salvo il dataset pulito dai valori indesiderati
dataset_cleaned.to_csv("cleaned_dataset.csv", index=False)


#**Operazione su *single_matches_dataset***

Facciamo adesso alcune operazioni specifiche sulle feature su un dataset composto unicamente da partite di singolo


In [260]:
columns_to_drop = ['match_format', 'player_id', 'player_2_id']

# Rimuovo dal dataset le partite di doppio e di team
dataset_singles = dataset_cleaned[dataset_cleaned["player_2_id"].isna()]
dataset_singles = dataset_singles[~dataset_singles["match_format"].isin(["T"])]

# Rimuovo le colonne non desiderate
dataset_singles = dataset_singles.drop(columns=columns_to_drop, axis=1)

# Traduco il campo players_gender in valore numerico
dataset_singles['players_gender'] = (dataset_singles['players_gender'] == 'M').astype(int)

dataset_singles

,players_gender,is_final,sets_to_win,opponent_sets_to_win,player_points,is_winner
50,1,1,4,4,"[1, 2, 3, 4, 5, 5, 5, 5, 6, 6, 6, 7, 7, 8, 9, ...",1
51,1,1,4,4,"[0, 0, 0, 0, 0, 1, 2, 3, 3, 4, 5, 5, 6, 6, 6, ...",0
52,1,1,3,4,"[0, 0, 1, 1, 1, 1, 2, 3, 3, 3, 4, 5, 6, 6, 7, ...",1
53,1,1,4,3,"[1, 2, 2, 3, 4, 5, 5, 5, 6, 7, 7, 7, 7, 8, 8, ...",0
54,1,1,3,3,"[1, 2, 2, 2, 3, 3, 4, 5, 6, 7, 8, 8, 9, 10, 10...",1
...,...,...,...,...,...,...
144189,1,0,2,2,"[0, 1, 1, 2, 2, 3, 4, 4, 4, 5, 5, 6, 6, 7, 8, ...",1
144190,1,0,2,1,"[1, 1, 2, 3, 3, 4, 5, 6, 6, 6, 7, 7, 7, 7, 8, ...",0
144191,1,0,1,2,"[0, 1, 1, 1, 2, 2, 2, 2, 3, 4, 4, 5, 6, 7, 7, ...",1
144192,1,0,1,1,"[1, 2, 2, 3, 3, 3, 3, 3, 3, 3, 4, 5, 5, 5, 6, 6]",0


Salvataggio del nuovo dataset

In [261]:
# Salvo il dataset finale delle partite di singolo
dataset_singles.to_csv("singles_matches_dataset.csv", index=False)


#**Operazione su *all_matches_dataset***

Facciamo adesso alcune operazioni specifiche sulle feature su un dataset composto da tutte le tipologie di partite


##**One-Hot Encoding**
In questo caso la colonna *players_gender* non potrà essere tradotta con un valore numerico binario poichè bisogna considerare il caso "X" in cui la partita è di doppio misto(quindi sia maschi che femmine). Quindi si dovrà utilizzare one hot encoding che trasformerà le seguenti combinazioni in:

*   M = [1,0,0]
*   W = [0,1,0]
*   X = [0,0,1]

Tradurre la colonna con (1, 2, 3),sarebbe utile **se volessimo assumere un ordine delle categorie** cosa che a noi non ci interessa in questo contesto.

In [262]:
dataset_all_matches = dataset_cleaned
# Applichiamo il One Hot Encoding sulla colonna 'players_gender'
encoder = OneHotEncoder(sparse_output=False)  # sparse_output=False per ottenere una matrice densa

# Trasformiamo i dati facendo One Hot Encoding sulla colonna players_gender
X_encoded = encoder.fit_transform(dataset_cleaned[['players_gender']])

# Otteniamo i nomi delle nuove colonne
encoded_columns = encoder.get_feature_names_out(['players_gender'])

dataset_cleaned[encoded_columns] = X_encoded

dataset_cleaned

,players_gender,is_final,match_format,sets_to_win,opponent_sets_to_win,player_id,player_2_id,player_points,is_winner,players_gender_M,players_gender_W,players_gender_X
0,W,1,D,3,3,112463,115009.0,"[0, 0, 1, 2, 2, 3, 4, 4, 4, 5, 6, 6, 6, 6, 6, ...",1,0.0,1.0,0.0
1,W,1,D,3,3,117294,109777.0,"[1, 2, 2, 2, 3, 3, 3, 4, 5, 5, 5, 6, 7, 8, 9, ...",0,0.0,1.0,0.0
2,W,1,D,3,2,112463,115009.0,"[1, 1, 1, 1, 2, 3, 4, 5, 6, 7, 7, 8, 9, 9, 9, ...",1,0.0,1.0,0.0
3,W,1,D,2,3,117294,109777.0,"[0, 1, 2, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 5, 6, ...",0,0.0,1.0,0.0
4,W,1,D,2,2,112463,115009.0,"[1, 1, 2, 2, 2, 3, 3, 4, 4, 5, 6, 6, 6, 6, 7, ...",1,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
144189,M,0,S,2,2,137238,NaN,"[0, 1, 1, 2, 2, 3, 4, 4, 4, 5, 5, 6, 6, 7, 8, ...",1,1.0,0.0,0.0
144190,M,0,S,2,1,132172,NaN,"[1, 1, 2, 3, 3, 4, 5, 6, 6, 6, 7, 7, 7, 7, 8, ...",0,1.0,0.0,0.0
144191,M,0,S,1,2,137238,NaN,"[0, 1, 1, 1, 2, 2, 2, 2, 3, 4, 4, 5, 6, 7, 7, ...",1,1.0,0.0,0.0
144192,M,0,S,1,1,132172,NaN,"[1, 2, 2, 3, 3, 3, 3, 3, 3, 3, 4, 5, 5, 5, 6, 6]",0,1.0,0.0,0.0


Anche in questo ordiniamo il database semplicemente rimuovendo la colonna *players_gender* e aggiungendo le altre 3 colonne


*   *players_gender_M*
*   *players_gender_W*
*   *players_gender_X*


In [263]:
# Rimuoviamo la colonna 'players_gender'
dataset_all_matches = dataset_all_matches.drop(columns=['players_gender'])

# Spostiamo le colonne codificate all'inizio
encoded_columns = ['players_gender_M', 'players_gender_W', 'players_gender_X']
dataset_all_matches = dataset_all_matches[encoded_columns + [col for col in dataset_all_matches.columns if col not in encoded_columns]]
dataset_all_matches[encoded_columns] = dataset_all_matches[encoded_columns].astype(int)

# Visualizziamo il risultato
dataset_all_matches

,players_gender_M,players_gender_W,players_gender_X,is_final,match_format,sets_to_win,opponent_sets_to_win,player_id,player_2_id,player_points,is_winner
0,0,1,0,1,D,3,3,112463,115009.0,"[0, 0, 1, 2, 2, 3, 4, 4, 4, 5, 6, 6, 6, 6, 6, ...",1
1,0,1,0,1,D,3,3,117294,109777.0,"[1, 2, 2, 2, 3, 3, 3, 4, 5, 5, 5, 6, 7, 8, 9, ...",0
2,0,1,0,1,D,3,2,112463,115009.0,"[1, 1, 1, 1, 2, 3, 4, 5, 6, 7, 7, 8, 9, 9, 9, ...",1
3,0,1,0,1,D,2,3,117294,109777.0,"[0, 1, 2, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 5, 6, ...",0
4,0,1,0,1,D,2,2,112463,115009.0,"[1, 1, 2, 2, 2, 3, 3, 4, 4, 5, 6, 6, 6, 6, 7, ...",1
...,...,...,...,...,...,...,...,...,...,...,...
144189,1,0,0,0,S,2,2,137238,NaN,"[0, 1, 1, 2, 2, 3, 4, 4, 4, 5, 5, 6, 6, 7, 8, ...",1
144190,1,0,0,0,S,2,1,132172,NaN,"[1, 1, 2, 3, 3, 4, 5, 6, 6, 6, 7, 7, 7, 7, 8, ...",0
144191,1,0,0,0,S,1,2,137238,NaN,"[0, 1, 1, 1, 2, 2, 2, 2, 3, 4, 4, 5, 6, 7, 7, ...",1
144192,1,0,0,0,S,1,1,132172,NaN,"[1, 2, 2, 3, 3, 3, 3, 3, 3, 3, 4, 5, 5, 5, 6, 6]",0


Effettuo le operazioni finali su all_matches_dataset

In [264]:
columns_to_drop = ['match_format', 'player_id', 'player_2_id']

# Rimuovo le colonne non desiderate
dataset_all_matches = dataset_all_matches.drop(columns=columns_to_drop, axis=1)

dataset_all_matches

,players_gender_M,players_gender_W,players_gender_X,is_final,sets_to_win,opponent_sets_to_win,player_points,is_winner
0,0,1,0,1,3,3,"[0, 0, 1, 2, 2, 3, 4, 4, 4, 5, 6, 6, 6, 6, 6, ...",1
1,0,1,0,1,3,3,"[1, 2, 2, 2, 3, 3, 3, 4, 5, 5, 5, 6, 7, 8, 9, ...",0
2,0,1,0,1,3,2,"[1, 1, 1, 1, 2, 3, 4, 5, 6, 7, 7, 8, 9, 9, 9, ...",1
3,0,1,0,1,2,3,"[0, 1, 2, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 5, 6, ...",0
4,0,1,0,1,2,2,"[1, 1, 2, 2, 2, 3, 3, 4, 4, 5, 6, 6, 6, 6, 7, ...",1
...,...,...,...,...,...,...,...,...
144189,1,0,0,0,2,2,"[0, 1, 1, 2, 2, 3, 4, 4, 4, 5, 5, 6, 6, 7, 8, ...",1
144190,1,0,0,0,2,1,"[1, 1, 2, 3, 3, 4, 5, 6, 6, 6, 7, 7, 7, 7, 8, ...",0
144191,1,0,0,0,1,2,"[0, 1, 1, 1, 2, 2, 2, 2, 3, 4, 4, 5, 6, 7, 7, ...",1
144192,1,0,0,0,1,1,"[1, 2, 2, 3, 3, 3, 3, 3, 3, 3, 4, 5, 5, 5, 6, 6]",0


Salvataggio del nuovo dataset

In [265]:
# Salvo il dataset finale di tutte le partite
dataset_all_matches.to_csv("all_matches_dataset.csv", index=False)

#**Modello**
Dobbiamo considerare tutte le istanze di un singolo punteggio per poter allenare il modello
Quindi per ciascuna istanza eseguiamo la seguente funzione:

In [271]:
def generate_score_sequence(player_points):
    points_list = player_points.split(', ')  # Separiamo i punteggi
    sequence = []
    for i in range(1, len(points_list) + 1):  # Creiamo la sequenza
        sequence.append(f"[{', '.join(points_list[:i])}]")  # Aggiungiamo i primi i numeri tra parentesi quadre e separati da virgola
    return sequence  # Restituiamo una lista di sequenze

dataset_singles['score_sequences'] = dataset_singles['player_points'].apply(generate_score_sequence)

dataset_singles = dataset_singles.explode('score_sequences', ignore_index=True)

dataset_singles

,players_gender,is_final,sets_to_win,opponent_sets_to_win,player_points,is_winner,score_sequences
0,1,1,4,4,"[1, 2, 3, 4, 5, 5, 5, 5, 6, 6, 6, 7, 7, 8, 9, ...",1,[[1]
1,1,1,4,4,"[1, 2, 3, 4, 5, 5, 5, 5, 6, 6, 6, 7, 7, 8, 9, ...",1,"[[1, 2]"
2,1,1,4,4,"[1, 2, 3, 4, 5, 5, 5, 5, 6, 6, 6, 7, 7, 8, 9, ...",1,"[[1, 2, 3]"
3,1,1,4,4,"[1, 2, 3, 4, 5, 5, 5, 5, 6, 6, 6, 7, 7, 8, 9, ...",1,"[[1, 2, 3, 4]"
4,1,1,4,4,"[1, 2, 3, 4, 5, 5, 5, 5, 6, 6, 6, 7, 7, 8, 9, ...",1,"[[1, 2, 3, 4, 5]"
...,...,...,...,...,...,...,...
1752967,1,0,1,1,"[0, 0, 1, 1, 2, 3, 4, 5, 6, 7, 7, 7, 8, 9, 9, 10]",1,"[[0, 0, 1, 1, 2, 3, 4, 5, 6, 7, 7, 7]"
1752968,1,0,1,1,"[0, 0, 1, 1, 2, 3, 4, 5, 6, 7, 7, 7, 8, 9, 9, 10]",1,"[[0, 0, 1, 1, 2, 3, 4, 5, 6, 7, 7, 7, 8]"
1752969,1,0,1,1,"[0, 0, 1, 1, 2, 3, 4, 5, 6, 7, 7, 7, 8, 9, 9, 10]",1,"[[0, 0, 1, 1, 2, 3, 4, 5, 6, 7, 7, 7, 8, 9]"
1752970,1,0,1,1,"[0, 0, 1, 1, 2, 3, 4, 5, 6, 7, 7, 7, 8, 9, 9, 10]",1,"[[0, 0, 1, 1, 2, 3, 4, 5, 6, 7, 7, 7, 8, 9, 9]"


Adesso applichiamo il dataset finale a cui è stato applicato padding e eleminazione dei casi con punteggi elevati

In [274]:
X_bin = dataset_singles[['players_gender', 'is_final', 'sets_to_win', 'opponent_sets_to_win']].values

# Funzione di pre-processing per player_points
def preprocess_player_points(points):
    # Rimuoviamo caratteri indesiderati e spazi
    cleaned_points = points.replace('[', '').replace(']', '').strip()
    # Convertiamo in lista di interi
    return list(map(int, cleaned_points.split(',')))

# Applichiamo la funzione di pre-processing alla colonna player_points
player_points = dataset_singles['player_points'].apply(preprocess_player_points)

# La variabile target
Y = dataset_singles['is_winner'].values

# Padding delle sequenze per avere la stessa lunghezza
max_length = max(len(seq) for seq in player_points)  # Lunghezza massima della sequenza
padded_player_points = [seq + [0] * (max_length - len(seq)) for seq in player_points]

# Unione delle caratteristiche binarie e della sequenza
X = [np.concatenate([x_bin, seq]) for x_bin, seq in zip(X_bin, padded_player_points)]


sequence_input_dim = len(dataset_singles['player_points'].iloc[0])  # La lunghezza della sequenza di player_points
sequence_length = max_length  # Lunghezza massima della sequenza
X_bin_input_dim = X_bin.shape[1]  # Numero di colonne in X_bin

# Definiamo gli input per il modello
sequence_input = Input(shape=(sequence_length,), dtype=tf.int32, name='sequence_input')  # Input sequenziale
bin_input = Input(shape=(X_bin_input_dim,), dtype=tf.float32, name='bin_input')  # Input variabili binarie

# Crea un layer di embedding per il sequenziale (opzionale, se le sequenze sono di tipo numerico, puoi ometterlo)
embedding_layer = Embedding(input_dim=100, output_dim=32, input_length=sequence_length)(sequence_input)  # Puoi variare i parametri

# Aggiungi un layer LSTM
lstm_layer = LSTM(64)(embedding_layer)  # Puoi regolare la dimensione dell'output dell'LSTM

# Aggiungi un layer Dropout per evitare overfitting
dropout_layer = Dropout(0.2)(lstm_layer)

# Combina l'output dell'LSTM con le variabili binarie
merged_layer = Concatenate()([dropout_layer, bin_input])

# Aggiungi un layer fully connected per la classificazione
dense_layer = Dense(64, activation='relu')(merged_layer)

# Aggiungi l'output finale
output_layer = Dense(1, activation='sigmoid')(dense_layer)  # 'sigmoid' per un problema di classificazione binaria

# Costruisci il modello
model = Model(inputs=[sequence_input, bin_input], outputs=output_layer)

# Compila il modello
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Visualizza la struttura del modello
model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ sequence_input            │ (None, 29)             │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_7 (Embedding)   │ (None, 29, 32)         │          3,200 │ sequence_input[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_7 (LSTM)             │ (None, 64)             │         24,832 │ embedding_7[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_7 (Dropout)       │ (None, 64)             │              0 │ lstm_7[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bin_input (InputLayer)    │ (None, 4)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_7             │ (None, 68)             │              0 │ dropout_7[0][0],       │
│ (Concatenate)             │                        │                │ bin_input[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_14 (Dense)          │ (None, 64)             │          4,416 │ concatenate_7[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_15 (Dense)          │ (None, 1)              │             65 │ dense_14[0][0]         │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 32,513 (127.00 KB)

 Trainable params: 32,513 (127.00 KB)

 Non-trainable params: 0 (0.00 B)

In [275]:

#########################
# Converting to numpy array per l'addestramento
X_bin = dataset_singles[['players_gender', 'is_final', 'sets_to_win', 'opponent_sets_to_win']].values
X_bin = np.array(X_bin)

# Le sequenze (padded)
X_seq = np.array(padded_player_points)

# Le etichette (target)
Y = dataset_singles['is_winner'].values

# Allenamento del modello
model.fit([X_seq, X_bin], Y, epochs=3, batch_size=64, validation_split=0.2)

Epoch 1/3
  130/21913 ━━━━━━━━━━━━━━━━━━━━ 10:22 29ms/step - accuracy: 0.6300 - loss: 0.6304

KeyboardInterrupt: 

In [ ]:
X_bin_test = np.array([
    [0, 0, 3, 2],
    [0, 1, 1, 1]
])

# X_seq specificato dall'utente
X_seq_test = [
    [0, 1, 2, 3, 4, 5, 6],  # Sequenza 1
    [0, 0, 0, 0, 1, 1, 1, 1, 1, 1]  # Sequenza 2
]

# Lunghezza massima delle sequenze (come durante l'addestramento)
max_length = 47

# Padding delle sequenze
X_seq_test_padded = np.array([seq + [0] * (max_length - len(seq)) for seq in X_seq_test])

# Stampa per verifica
print("X_seq_test_padded:")
print(X_seq_test_padded)
print("\nX_bin_test:")
print(X_bin_test)

# Ora possiamo utilizzare questi dati per fare previsioni
predictions = model.predict([X_seq_test_padded, X_bin_test])

# Stampiamo le previsioni
print("Predictions:")
print(predictions)

# Interpretazione delle previsioni
for i, pred in enumerate(predictions):
    print(f"Campione {i+1}: Probabilità di vittoria = {pred[0]:.2f}")